# ETL

Archivo para descargar los datos del tipo de cambio y de la tasa de interés de la API de Banxico y la inflación de la API del INEGI y almacenarlos en Amazon S3.

In [19]:
# Cargamos los paquetes necesarios
from sie_banxico import SIEBanxico
from INEGIpy import Indicadores
import pandas as pd
import boto3
import yaml

## Extract

In [2]:
# Abrir yaml
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Leémos el token proporcionado por Banxico para acceder a su API
token_bmx = config['etl']['token_bmx']
# Accedemos a las series de tipo de cambio y tasa de interés a través de la API
api = SIEBanxico(token = token_bmx, id_series = ['SF63528', 'SF43783'], language = 'en')
# api.get_metadata()
data = api.get_timeseries()
# Convertimos la serie de tipo de cambio en un dataframe para su manipulación
tbl_tipo_de_cambio_raw = pd.DataFrame(data['bmx']['series'][0]['datos'])
# Convertimos la serie de tasa de interés en un dataframe para su manipulación
tbl_tasa_de_interes_raw = pd.DataFrame(data['bmx']['series'][1]['datos'])

# Leémos el token proporcionado por el Inegi para acceder a su API
token_inegi = config['etl']['token_inegi']
# Accedemos al banco de indicadores del Inegi
inegi = Indicadores(token_inegi)
# Creamos un dataframe con los datos de inflación quincenal
tbl_inflacion_raw = inegi.obtener_df(indicadores = "628229", 
                      nombres = 'Inflación quincenal')

## Transform

In [12]:
# Definimos un diccionario con los nombres limpios para la tabla tbl_tipo_de_cambio_raw
DICT_NAMES_TIPO_DE_CAMBIO_TABLE = {
    'fecha': 'date',
    'dato': 'tipo_de_cambio'
}
# Cambiamos el nombre de las columnas en la tabla 
tbl_tipo_de_cambio_raw = tbl_tipo_de_cambio_raw.rename(columns = DICT_NAMES_TIPO_DE_CAMBIO_TABLE)
# Cambiamos el formato de la columna date a datetime
tbl_tipo_de_cambio_raw['date'] = pd.to_datetime(tbl_tipo_de_cambio_raw['date'], format='%d/%m/%Y')
# Guardamos la tabla tbl_tipo_de_cambio_raw en un csv
tbl_tipo_de_cambio_raw.to_csv("./datos/tipo_de_cambio.csv", index=False)

# Definimos un diccionario con los nombres limpios para la tabla tbl_tasa_de_interes_raw
DICT_NAMES_TASA_DE_INTERES_TABLE = {
    'fecha': 'date',
    'dato': 'tasa_de_interes'
}
# Cambiamos el nombre de las columnas en la tabla tbl_tasa_de_interes_raw
tbl_tasa_de_interes_raw = tbl_tasa_de_interes_raw.rename(columns = DICT_NAMES_TASA_DE_INTERES_TABLE)
# Cambiamos el formato de la columna date a datetime
tbl_tasa_de_interes_raw['date'] = pd.to_datetime(tbl_tasa_de_interes_raw['date'], format='%d/%m/%Y')
# Guardamos la tabla tbl_tasa_de_interes_raw en un csv
tbl_tasa_de_interes_raw.to_csv("./datos/tasa_de_interes.csv", index=False)

# Defininimos un diccionario con los nombres limpios para la tabla tbl_inflacion_raw
DICT_NAMES_INFLACION_TABLE = {
    'Inflación quincenal': 'inflacion'
}
# Convertimos los row_names con la información de la fecha en columna
tbl_inflacion_raw = tbl_inflacion_raw.reset_index().rename(columns={"index":"date"})		
# Cambiamos el nombre de las columnas en la tabla tbl_inflacion_raw
tbl_inflacion_raw = (tbl_inflacion_raw
                     .rename(columns = DICT_NAMES_INFLACION_TABLE)
)
# Cambiamos el formato de la columna date a datetime
tbl_inflacion_raw['date'] = pd.to_datetime(tbl_inflacion_raw['date'], format='%Y-%m-%d')
# Guardamos la tabla tbl_inflacion en un csv
tbl_inflacion_raw.to_csv("./datos/inflacion.csv", index=False)

## Load

In [15]:
# Abrimos un cliente de S3
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')

# Cargamos los tres archivos a S3
BUCKET_NAME = "itam-analytics-javier"
s3.upload_file(Filename="./datos/tasa_de_interes.csv", Bucket=BUCKET_NAME, Key="tarea6/raw/tasa_de_interes/tasa_de_interes.csv")
s3.upload_file(Filename="./datos/tipo_de_cambio.csv", Bucket=BUCKET_NAME, Key="tarea6/raw/tipo_de_cambio/tipo_de_cambio.csv")
s3.upload_file(Filename="./datos/inflacion.csv", Bucket=BUCKET_NAME, Key="tarea6/raw/inflacion/inflacion.csv")